## UAV fleet Simulation 

This is for testing propouses

In [1]:
# Custom modules
from config import Color
from helpers import clean
from helpers.change_coordinates import rel_to_abs
from mavlink.customtypes.location import ENU, ENUPose, GRAPose
from plan import Plan
from simulators import QGC, ConfigGazebo, Gazebo, Simulator

clean()

## Create Plans

In [2]:
offset = ENUPose(-1,1,0,0)  # east, north, up, heading 
origin = GRAPose(-35.3633245, 149.1652241, 0, 0)
rel_path = Plan.create_square_path(side_len=5, alt=5,clockwise=False)
plan = Plan.basic(wps=rel_path, wp_margin=0.5)

## Visualization Parameters

In [3]:
home = ENU(*offset[:3])
abs_path = [rel_to_abs(pos, home, cls=ENU) for pos in rel_path]
color = Color.BLUE
markers = [
	ConfigGazebo.create_markertraj(traj=abs_path, color=color)
]
Gazebo.plot_3d_interactive(
	markers=markers,
	title="Simulation Waypoints",
	frames=(0.2, 0.2, 0.6),
	ground=-0.05
)

## Choose Simulator

In [4]:



simulator = QGC(offsets=[offset], plans=[plan], origin=origin)

gazebo_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway.world",
                           models=[("iris",color)],
                           marker_trajs = markers)
simulator = Gazebo(offsets=[offset],plans=[plan],config=gazebo_config)

simulator = Simulator(offsets=[offset], plans=[plan],visible_terminals=False,verbose=2)

Launch Simulator

In [5]:
orac = simulator.launch(gcs_sysids={'blue 🟦':[1]})

🙈 Running without visualization.
🚀 ArduPilot SITL vehicle 1 launched (PID 2386092)
🚀 UAV logic for vehicle 1 launched (PID 2386093)
🚀 Proxy for vehicle 1 launched (PID 2386094)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
System id: 1
System id: 1
[Errno 111] Connection refused sleeping
SIM_VEHICLE: Start
SIM_VEHICLE: Killing tasks
SIM_VEHICLE: Starting up at SITL location
SIM_VEHICLE: Using defaults from (../../ardupilot/Tools/autotest/default_params/copter.parm)
SIM_VEHICLE: Adding parameters from (/home/abeldg/uav-cyber-sim/params/vehicle.parm)
Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=1
Suggested EK3_DRAG_BCOEF_* = 17.209, EK3_DRAG_MCOEF = 0.209
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9005
Waiting for connection ....


bind port 5760 for SERIAL0
SERIAL0 on TCP port 5760


Connection on serial port 5760

🚀 Starting Proxy 1
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Home: -35.363262 149.165237 alt=584.000000m hdg=353.000000
Smoothing reset at 0.001
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,/home/abeldg/uav-cyber-sim/par

bind port 5762 for SERIAL1
SERIAL1 on TCP port 5762
bind port 5763 for SERIAL2
SERIAL2 on TCP port 5763
validate_structures:528: Validating structures
Waiting for internal clock bits to be set (current=0x00)


🔗 UAV logic 1 is connected to Oracle ⚪
✅ Heartbeat received

🚀 Starting Vehicle 1 logic
Vehicle 1: ▶️ Plan Started: 📋 basic
🚀 GCS blue 🟦 launched (PID 2386270)


## Execute Plan

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 1: ✅ Step Done: Check disarmed
📡 Requested message EKF_STATUS_REPORT at 1.00 Hz
Vehicle 1: ▶️ Step Started: Check EKF status
🔗 UAV logic 1 is connected
SIM_VEHICLE: Waiting for SITL to exit
⌛ Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
⌛ Waiting for EKF to be 

📤 GCS ← UAV 1: Sending DONE (attempt 2)
📤 GCS ← UAV 1: Sending DONE (attempt 3)
📤 GCS ← UAV 1: Sending DONE (attempt 4)
✅ Vehicle 1 completed its mission
✅ All UAVs assigned to GCS blue 🟦 have completed their mission.
✅ ACK received. DONE message acknowledged.
❎ Vehicle 1 logic stopped.
Connection reset or closed by peer on TCP socket
💾 Trajectories saved to 'trajectories_blue 🟦.pkl'.
❎ Proxy 1 stopped.
Closed connection on SERIAL0
